In [29]:
from google.cloud import bigquery

In [2]:
client = bigquery.Client()

/Users/shinji/opt/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/shinji/opt/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google

In [5]:
dataset_ref = client.dataset("openaq", project="bigquery-public-data")

In [6]:
dataset = client.get_dataset(dataset_ref)

In [7]:
tables = list(client.list_tables(dataset))

In [8]:
for table in tables:
    print(table.table_id)

global_air_quality


In [9]:
table_ref = dataset_ref.table("global_air_quality")

In [10]:
table = client.get_table(table_ref)

In [11]:
client.list_rows(table, max_results=5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25


In [12]:
query = """
SELECT
  city
FROM 
  `bigquery-public-data.openaq.global_air_quality`
WHERE 
  country = "US"
"""

In [31]:
client = bigquery.Client()

/Users/shinji/opt/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/shinji/opt/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google

In [14]:
query_job = client.query(query)

In [16]:
us_cities = query_job.to_dataframe()

In [18]:
us_cities.city.value_counts().head()

Phoenix-Mesa-Scottsdale                     88
Houston                                     82
Los Angeles-Long Beach-Santa Ana            68
Riverside-San Bernardino-Ontario            60
New York-Northern New Jersey-Long Island    60
Name: city, dtype: int64

In [32]:
query = """
SELECT 
  score,
  title
FROM
`bigquery-public-data.hacker_news.full`
WHERE
  type = "job" 
"""

In [33]:
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

In [34]:
dry_run_query_job = client.query(query, job_config=dry_run_config)

In [35]:
print(f"This query will process {dry_run_query_job.total_bytes_processed} bytes.")

This query will process 414769269 bytes.


In [36]:
ONE_MB = 1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_MB)
safe_query_job = client.query(query, job_config=safe_config)
safe_query_job.to_dataframe()

BadRequest: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/test-chematels/queries/836f8841-4b2b-490e-86ce-f482afb869e2?maxResults=0&location=US: Query exceeded limit for bytes billed: 1000000. 415236096 or higher required.

(job ID: 836f8841-4b2b-490e-86ce-f482afb869e2)

      -----Query Job SQL Follows-----       

    |    .    |    .    |    .    |
   1:
   2:SELECT 
   3:  score,
   4:  title
   5:FROM
   6:`bigquery-public-data.hacker_news.full`
   7:WHERE
   8:  type = "job" 
    |    .    |    .    |    .    |

In [40]:
ONE_GB = 1000*1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_GB)
safe_query_job = client.query(query, job_config=safe_config)
job_post_scores = safe_query_job.to_dataframe()
job_post_scores.score.mean()

1.8903448830291825